<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/5.12.%20%E7%A8%A0%E5%AF%86%E8%BF%9E%E6%8E%A5%E7%BD%91%E7%BB%9C(DenseNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##5.12. 稠密连接网络（DenseNet）
ResNet中的跨层连接设计引申出了数个后续工作。本节我们介绍其中的一个：稠密连接网络（DenseNet） [1]。 它与ResNet的主要区别如图5.10所示。
<div align=center><img src="https://zh.gluon.ai/_images/densenet.svg" width="500"/></div>

<center>图 5.10 ResNet（左）与DenseNet（右）在跨层连接上的主要区别：使用相加和使用连结</center>

图5.10中将部分前后相邻的运算抽象为模块 $A$ 和模块 $B$ 。与ResNet的主要区别在于，DenseNet里模块 $B$ 的输出不是像ResNet那样和模块 $A$ 的输出相加，而是在通道维上连结。这样模块 $A$ 的输出可以直接传入模块 $B$ 后面的层。在这个设计里，模块 $A$ 直接跟模块 $B$ 后面的所有层连接在了一起。这也是它被称为“稠密连接”的原因。

DenseNet的主要构建模块是稠密块（dense block）和过渡层（transition layer）。前者定义了输入和输出是如何连结的，后者则用来控制通道数，使之不过大。

###5.12.1. 稠密块
DenseNet使用了ResNet改良版的“批量归一化、激活和卷积”结构（参见上一节的练习），我们首先在conv_block函数里实现这个结构。

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from tensorflow import data
from tensorflow.keras import losses,optimizers
import time
import numpy as np
from tensorflow import image
import os
import sys

In [0]:
# !pip install tensorflow==2.0.0
tf.__version__

'2.0.0'

稠密块由多个conv_block组成，每块使用相同的输出通道数。但在前向计算时，我们将每块的输入和输出在通道维上连结。

In [0]:
def conv_block(num_channels):
  blk=Sequential([
      layers.BatchNormalization(),
      layers.Activation('relu'),
      layers.Convolution2D(num_channels,kernel_size=3,padding='same')
  ])
  return blk

In [0]:
# tf.enable_eager_execution() #启用动态图计算，tf2.x不用调用该函数

In [0]:
class DenseBlock(keras.Model):
  def __init__(self,num_convs,num_channels,**kwargs):
    super(DenseBlock,self).__init__()
    self.num_convs=num_convs #卷积模块数量
    self.conv_blocks=[] #存放所有的卷积模块
    for _ in range(self.num_convs):
      self.conv_blocks.append(conv_block(num_channels))

  def call(self,X):
    #遍历所有的卷积模块并传入上层输出作为输入
    for i in range(int(self.num_convs)):
      output=self.conv_blocks[i](X)
      X=tf.concat([X,output],axis=-1)#在通道维上将输入和输出连接
    return X

在下面的例子中，我们定义一个有2个输出通道数为10的卷积块。使用通道数为3的输入时，我们会得到通道数为 $3 + 2 \times 10 =23$ 的输出。卷积块的通道数控制了输出通道数相对于输入通道数的增长，因此也被称为增长率（growth rate）。

In [0]:
blk=DenseBlock(2,10)
X=tf.random.uniform(shape=(4,8,8,3))
Y=blk(X)
Y.shape

TensorShape([4, 8, 8, 23])

###5.12.2. 过渡层
由于每个稠密块都会带来通道数的增加，使用过多则会带来过于复杂的模型。过渡层用来控制模型复杂度。它通过$ 1\times1$ 卷积层来减小通道数，并使用步幅为2的平均池化层减半高和宽，从而进一步降低模型复杂度。

In [0]:
def transition_block(num_channels):
  blk=Sequential([
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Convolution2D(num_channels,kernel_size=1),
    layers.AveragePooling2D(pool_size=2,strides=2)                  
  ])
  return blk

对上一个例子中稠密块的输出使用通道数为10的过渡层。此时输出的通道数减为10，高和宽均减半。

In [0]:
blk=transition_block(10)
blk(Y).shape

TensorShape([4, 4, 4, 10])

###5.12.3. DenseNet模型
我们来构造DenseNet模型。DenseNet首先使用同ResNet一样的单卷积层和最大池化层。\
类似于ResNet接下来使用的4个残差块，DenseNet使用的是4个稠密块。同ResNet一样，我们可以设置每个稠密块使用多少个卷积层。这里我们设成4，从而与上一节的ResNet-18保持一致。稠密块里的卷积层通道数（即增长率）设为32，所以每个稠密块将增加128个通道。

ResNet里通过步幅为2的残差块在每个模块之间减小高和宽。这里我们则使用过渡层来减半高和宽，并减半通道数。

In [0]:
net=Sequential(
    [
     layers.Convolution2D(64,kernel_size=7,strides=2,padding='same',input_shape=(224,224,1)),
     layers.BatchNormalization(),
     layers.Activation('relu'),
     layers.MaxPool2D(pool_size=3,strides=2,padding='same')
    ]
)
num_channels,growth_rate=64,32 #num_channels为当前的通道数
num_convs_in_dense_blocks=[4,4,4,4]
for i,num_convs in enumerate(num_convs_in_dense_blocks):
  net.add(DenseBlock(num_convs,growth_rate))
  #上一个稠密块的输出通道数
  num_channels+=num_convs*growth_rate
  #在稠密块之间加入通道数减半的过渡层
  if i!=len(num_convs_in_dense_blocks)-1:
    num_channels //2
    net.add(transition_block(num_channels))

net.add(layers.BatchNormalization())
net.add(layers.Activation('relu'))
net.add(layers.GlobalAveragePooling2D())
net.add(layers.Dense(10))

###5.12.4. 获取数据并训练模型
由于这里使用了比较深的网络，本节里我们将输入高和宽从224降到96来简化计算。

In [0]:
lr,num_epochs,batch_size=0.05,5,128
optimizer=optimizers.SGD(learning_rate=lr)
loss=losses.SparseCategoricalCrossentropy(from_logits=True)
buffer_size=1000
def load_data_fashion_mnist(batch_size,buffer_size):
  (x_train,y_train),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()
  x_train=x_train[:,:,:,np.newaxis]#将三维张量增加一个channel维
  x_test=x_test[:,:,:,np.newaxis]
  train_iter=data.Dataset.from_tensor_slices((x_train,y_train)).map(lambda x,y:(x/255,y)).shuffle(buffer_size).batch(batch_size).map(lambda x,y:(image.resize(images=x,size=(224,224)),y))
  test_iter=data.Dataset.from_tensor_slices((x_test,y_test)).map(lambda x,y:(x/255,y)).batch(batch_size).map(lambda x,y:(image.resize(images=x,size=(224,224)),y))
  return train_iter,test_iter
train_iter,test_iter=load_data_fashion_mnist(batch_size=batch_size,buffer_size=buffer_size)

, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separate expression.
, which must contain a single lambda with matching signature. To avoid ambiguity, define each lambda in a separ

In [0]:
net.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])
history=net.fit_generator(train_iter,validation_data=test_iter,epochs=num_epochs)

In [0]:
X=tf.random.uniform(shape=(1,224,224,1))
for layer in net.layers:
  X=layer(X)
  print(layer.name,X.shape)

conv2d_3 (1, 112, 112, 64)
batch_normalization_3 (1, 112, 112, 64)
activation_3 (1, 112, 112, 64)
max_pooling2d (1, 56, 56, 64)
dense_block_2 (1, 56, 56, 192)
sequential_8 (1, 28, 28, 192)
dense_block_3 (1, 28, 28, 320)
sequential_13 (1, 14, 14, 320)
dense_block_4 (1, 14, 14, 448)
sequential_18 (1, 7, 7, 448)
dense_block_5 (1, 7, 7, 576)
batch_normalization_23 (1, 7, 7, 576)
activation_23 (1, 7, 7, 576)
global_average_pooling2d (1, 576)
dense (1, 10)


In [0]:
net.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 112, 112, 64)      3200      
_________________________________________________________________
batch_normalization_3 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
activation_3 (Activation)    (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0         
_________________________________________________________________
dense_block_2 (DenseBlock)   (None, 56, 56, 192)       130944    
_________________________________________________________________
sequential_8 (Sequential)    (None, 28, 28, 192)       37824     
_________________________________________________________________
dense_block_3 (DenseBlock)   (None, 28, 28, 320)      

###5.12.5. 小结
* 在跨层连接上，不同于ResNet中将输入与输出相加，DenseNet在通道维上连结输入与输出。
DenseNet的主要构建模块是稠密块和过渡层。